In [1]:
from keras.optimizers import *
from tensorflow import keras
from keras.models import *
from keras.layers.core import Dense, Dropout, Activation
from keras.layers import *
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import ImageDataGenerator
import pickle
import os
import numpy as np

In [2]:
class cnn():
    def __init__(self, data_path, load_history, img_rows = 32, img_cols = 32):
        self.img_rows = img_rows
        self.img_cols = img_cols
        self.data_path = data_path
        self.num_class = 100
        self.verbose = True
        self.load_history = load_history

    def data_init(self):
        with open(self.data_path + '/train', mode='rb') as file:
            batch = pickle.load(file, encoding='latin1')
            self.train_data = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
            self.train_label = batch['fine_labels']
            self.train_data = np.array((self.train_data))
            self.train_label = np.array((self.train_label))
            self.train_label = to_categorical(self.train_label, 100)
        with open(self.data_path + '/test', mode='rb') as file:
            batch = pickle.load(file, encoding='latin1')
            self.test_data = batch['data'].reshape((len(batch['data']), 3, 32, 32)).transpose(0, 2, 3, 1)
            self.test_label = batch['fine_labels']
            self.test_data = np.array((self.test_data))
            self.test_label = np.array((self.test_label))
            self.test_label = to_categorical(self.test_label, 100)

    def get_base(self):
        self.model = Sequential()
        self.model.add(Conv2D(16, 7, activation='relu', padding='valid', kernel_initializer='he_normal', input_shape = (32,32,3)))
        self.model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
        self.model.add(Conv2D(32, 5, activation='relu', padding='valid', kernel_initializer='he_normal'))
        self.model.add(MaxPooling2D(pool_size=(2, 2), strides=2))
        self.model.add(Flatten())
        self.model.add(Dense(128, activation='relu'))
        self.model.add(Dense(100, activation=None))
        # if os.path.exists('./model/base.h5') and self.load_history:
        #     self.model.load_weights('./model/base.h5',by_name=True,skip_mismatch=True)

    def conv_block(self, inputs, filter_num, reduction_ratio, stride=1, name=None, atten = False):
        x = inputs
        x = Conv2D(filter_num[0], (1, 1), strides=stride, padding='same', name=name + '_conv1')(x)
        x = BatchNormalization(axis=3, name=name + '_bn1')(x)
        x = Activation('relu', name=name + '_relu1')(x)

        x = Conv2D(filter_num[1], (3, 3), strides=1, padding='same', name=name + '_conv2')(x)
        x = BatchNormalization(axis=3, name=name + '_bn2')(x)
        x = Activation('relu', name=name + '_relu2')(x)

        x = Conv2D(filter_num[2], (1, 1), strides=1, padding='same', name=name + '_conv3')(x)
        x = BatchNormalization(axis=3, name=name + '_bn3')(x)
        if atten:
            # Channel Attention
            avgpool = GlobalAveragePooling2D(name=name + '_channel_avgpool')(x)
            maxpool = GlobalMaxPool2D(name=name + '_channel_maxpool')(x)
            # Shared MLP
            Dense_layer1 = Dense(filter_num[2] // reduction_ratio, activation='relu', name=name + '_channel_fc1')
            Dense_layer2 = Dense(filter_num[2], activation='relu', name=name + '_channel_fc2')
            avg_out = Dense_layer2(Dense_layer1(avgpool))
            max_out = Dense_layer2(Dense_layer1(maxpool))

            channel = add([avg_out, max_out])
            channel = Activation('sigmoid', name=name + '_channel_sigmoid')(channel)
            channel = Reshape((1, 1, filter_num[2]), name=name + '_channel_reshape')(channel)
            channel_out = tf.multiply(x, channel)

            # Spatial Attention
            avgpool = tf.reduce_mean(channel_out, axis=3, keepdims=True, name=name + '_spatial_avgpool')
            maxpool = tf.reduce_max(channel_out, axis=3, keepdims=True, name=name + '_spatial_maxpool')
            spatial = Concatenate(axis=3)([avgpool, maxpool])

            spatial = Conv2D(1, (7, 7), strides=1, padding='same', name=name + '_spatial_conv2d')(spatial)
            spatial_out = Activation('sigmoid', name=name + '_spatial_sigmoid')(spatial)

            CBAM_out = tf.multiply(channel_out, spatial_out)

            # residual connection
            r = Conv2D(filter_num[2], (1, 1), strides=stride, padding='same', name=name + '_residual')(inputs)
            x = add([CBAM_out, r])
        else:
            x = Activation('relu')(x)
            X_shortcut = Conv2D(filters=filter_num[2], kernel_size=(1, 1), strides=stride, padding='valid',
                                name=name + 'short1')(inputs)
            X_shortcut = BatchNormalization(axis=3, name=name + 'short2')(X_shortcut)
            x = add([x, X_shortcut])
        x = Activation('relu', name=name + '_relu3')(x)

        return x

    def build_block(self, x, filter_num, blocks, reduction_ratio=16, stride=1, name=None, atten = False):
        x = self.conv_block(x, filter_num, reduction_ratio, stride, name=name, atten = atten)
        for i in range(1, blocks):
            x = self.conv_block(x, filter_num, reduction_ratio, stride=1, name=name + '_block' + str(i), atten = atten)
        return x




    def get_model2(self, Netname, nb_classes, atten = False):
        layers_dims = [2, 2, 2, 2]

        filter_block1 = [64, 64, 256]
        filter_block2 = [128, 128, 512]
        filter_block3 = [256, 256, 1024]
        filter_block4 = [512, 512, 2048]

        # Reduction ratio in four blocks
        SE_reduction = [16, 16, 16, 16]

        inpt = Input(shape=(32, 32, 3))
        x  = UpSampling2D((5,5))(inpt)
        # stem block
        x = Conv2D(64, (7, 7), strides=(2, 2), padding='same', name='stem_conv')(x)
        x = BatchNormalization(axis=3, name='stem_bn')(x)
        x = Activation('relu', name='stem_relu')(x)
        x = MaxPooling2D((3, 3), strides=(2, 2), padding='same', name='stem_pool')(x)
        # convolution block
        x = self.build_block(x, filter_block1, layers_dims[0], SE_reduction[0], name='conv1', atten = atten)
        x = self.build_block(x, filter_block2, layers_dims[1], SE_reduction[1], stride=2, name='conv2', atten = atten)
        x = self.build_block(x, filter_block3, layers_dims[2], SE_reduction[2], stride=2, name='conv3', atten = atten)
        x = self.build_block(x, filter_block4, layers_dims[3], SE_reduction[3], stride=2, name='conv4', atten = atten)
        # top layer
        x = GlobalAveragePooling2D(name='top_layer_pool')(x)
        x = Dense(self.num_class, activation=None, name='fc')(x)

        self.model = Model(inpt, x, name=Netname)


    def fit(self, epoch = 1000, batch_size = 32, model_ = 'base'):
        model_checkpoint1 = ModelCheckpoint('./model/' + model_ + '.h5',
            monitor='val_loss', verbose=1, save_best_only=True)  
        self.model.compile(optimizer = Adam(lr=8e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-9),
                                loss = keras.losses.CategoricalCrossentropy(from_logits = True),
                                metrics=['accuracy'])
        self.history = self.model.fit(x=self.train_data, y=self.train_label,
                                      validation_data=(self.test_data, self.test_label),
                                      epochs=epoch, batch_size=batch_size, verbose=self.verbose,
                                      callbacks=[model_checkpoint1])

    def aug_fit(self, epoch = 1000, batch_size = 16, atten = False, model_ = 'base'):
        datagen = ImageDataGenerator(
            featurewise_center = True,
            featurewise_std_normalization = True,
            zca_epsilon=1e-06,
            rotation_range=10, #
            width_shift_range=0.1, 
            height_shift_range=0.1, 
            brightness_range=None,
            shear_range=0.1,
            zoom_range=0.1, 
            channel_shift_range=0.1, 
            fill_mode='nearest', 
            cval=0.1,
            horizontal_flip=True,
            vertical_flip=True)

        datagen.fit(np.concatenate([self.train_data, self.test_data], axis=0))
        aug_data = datagen.flow(self.train_data, self.train_label, batch_size = batch_size, shuffle = True)
        atten_n  = '_atten' if atten else ''
        model_checkpoint1 = ModelCheckpoint('./model/'+ model_ + atten_n + '.h5',
            monitor='val_loss', verbose=1, save_best_only=True) 

        self.model.compile(optimizer = Adam(lr=8e-5, beta_1=0.9, beta_2=0.999, epsilon=1e-9),
                                loss = keras.losses.CategoricalCrossentropy(from_logits = True),
                                metrics=['accuracy'])
        self.history = self.model.fit(aug_data,
                                      validation_data=datagen.flow(self.test_data, self.test_label),
                                      epochs=epoch, batch_size=batch_size, verbose=self.verbose,
                                      callbacks=[model_checkpoint1])

    def acc_test(self, model_ = 'base'):
        datagen = ImageDataGenerator(
            featurewise_center = True,
            featurewise_std_normalization = True,
            zca_epsilon=1e-06,
            rotation_range=10, #
            width_shift_range=0.1, 
            height_shift_range=0.1, 
            brightness_range=None,
            shear_range=0.1,
            zoom_range=0.1, 
            channel_shift_range=0.1, 
            fill_mode='nearest', 
            cval=0.1,
            horizontal_flip=True,
            vertical_flip=True)
            
        datagen.fit(np.concatenate([self.train_data, self.test_data], axis=0))
        if os.path.exists('./model/base.h5') and model_ == 'base':
            self.model = load_model('./model/base.h5')
            score = self.model.evaluate(self.test_data, self.test_label, verbose = 0 )
        elif os.path.exists('./model/improved.h5') and model_ == 'improved':
            self.model = load_model('./model/improved.h5')
            score = self.model.evaluate(datagen.flow(self.test_data, self.test_label), verbose = 0 )
        elif os.path.exists('./model/improved_atten.h5') and model_ == 'attention':
            self.model = load_model('./model/improved_atten.h5')
            score = self.model.evaluate(datagen.flow(self.test_data, self.test_label), verbose = 0 )
        
        print(model_ + " model loss: %.6f - acc: %.6f" % (score[0], score[1]))


mycnn = cnn('./data/cifar-100-python', False)

# load data

In [3]:
mycnn.data_init()

# get base model

In [4]:
# fucntion to create base model
mycnn.get_base()
mycnn.model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 16)        2368      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 16)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 9, 9, 32)          12832     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 4, 4, 32)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 128)               6

In [37]:
mycnn.fit(epoch = 20, batch_size =32, model_= 'base') #epoch is 50 in the report but reduce to 20 when submitted.

Epoch 1/50
1558/1563 [============================>.] - ETA: 0s - loss: 15.4343 - accuracy: 0.0107
Epoch 1: val_loss improved from inf to 4.62664, saving model to ./model\base.h5
1563/1563 [==============================] - 7s 4ms/step - loss: 15.4032 - accuracy: 0.0107 - val_loss: 4.6266 - val_accuracy: 0.0104
Epoch 2/50
1551/1563 [============================>.] - ETA: 0s - loss: 4.6226 - accuracy: 0.0096
Epoch 2: val_loss improved from 4.62664 to 4.60939, saving model to ./model\base.h5
1563/1563 [==============================] - 7s 4ms/step - loss: 4.6226 - accuracy: 0.0096 - val_loss: 4.6094 - val_accuracy: 0.0100
Epoch 3/50
1550/1563 [============================>.] - ETA: 0s - loss: 4.6088 - accuracy: 0.0097
Epoch 3: val_loss improved from 4.60939 to 4.60702, saving model to ./model\base.h5
1563/1563 [==============================] - 6s 4ms/step - loss: 4.6089 - accuracy: 0.0097 - val_loss: 4.6070 - val_accuracy: 0.0102
Epoch 4/50
1551/1563 [============================>.] - E

In [ ]:
mycnn.acc_test('base')

# aug and attention

In [44]:
#fucntion to create imroved model, if the atten is True, the model will add attention approach
mycnn.get_base()
mycnn.aug_fit(epoch = 20, batch_size = 32, model_= 'improved') 

Epoch 1/20
1562/1563 [============================>.] - ETA: 0s - loss: 4.3969 - accuracy: 0.0443
Epoch 1: val_loss improved from inf to 4.16623, saving model to ./model\base.h5
1563/1563 [==============================] - 31s 19ms/step - loss: 4.3967 - accuracy: 0.0443 - val_loss: 4.1662 - val_accuracy: 0.0682
Epoch 2/20
1563/1563 [==============================] - ETA: 0s - loss: 4.0544 - accuracy: 0.0881
Epoch 2: val_loss improved from 4.16623 to 3.96820, saving model to ./model\base.h5
1563/1563 [==============================] - 30s 19ms/step - loss: 4.0544 - accuracy: 0.0881 - val_loss: 3.9682 - val_accuracy: 0.0967
Epoch 3/20
1561/1563 [============================>.] - ETA: 0s - loss: 3.8972 - accuracy: 0.1089
Epoch 3: val_loss improved from 3.96820 to 3.85345, saving model to ./model\base.h5
1563/1563 [==============================] - 29s 19ms/step - loss: 3.8972 - accuracy: 0.1089 - val_loss: 3.8535 - val_accuracy: 0.1163
Epoch 4/20
1561/1563 [============================>.]

In [45]:
mycnn.get_model2('ResNet', 100)
mycnn.fit(epoch = 5, model_= 'improved') #epoch is 20 in the report but reduce to 5 when submitted.

Epoch 1/20
1562/1563 [============================>.] - ETA: 0s - loss: 3.5227 - accuracy: 0.1698
Epoch 1: val_loss improved from inf to 3.16425, saving model to ./model\improved.h5
1563/1563 [==============================] - 102s 64ms/step - loss: 3.5228 - accuracy: 0.1698 - val_loss: 3.1642 - val_accuracy: 0.2286
Epoch 2/20
1562/1563 [============================>.] - ETA: 0s - loss: 2.7211 - accuracy: 0.3123
Epoch 2: val_loss improved from 3.16425 to 3.04120, saving model to ./model\improved.h5
1563/1563 [==============================] - 102s 65ms/step - loss: 2.7212 - accuracy: 0.3124 - val_loss: 3.0412 - val_accuracy: 0.2686
Epoch 3/20
1562/1563 [============================>.] - ETA: 0s - loss: 2.2158 - accuracy: 0.4185
Epoch 3: val_loss improved from 3.04120 to 2.57776, saving model to ./model\improved.h5
1563/1563 [==============================] - 103s 66ms/step - loss: 2.2156 - accuracy: 0.4186 - val_loss: 2.5778 - val_accuracy: 0.3431
Epoch 4/20
1562/1563 [================

In [46]:
mycnn.get_model2('ResNet', 100)
mycnn.aug_fit(epoch = 5, model_= 'improved') #epoch is 20 in the report but reduce to 5 when submitted.

Epoch 1/20
3124/3125 [============================>.] - ETA: 0s - loss: 3.7976 - accuracy: 0.1240
Epoch 1: val_loss improved from inf to 3.57285, saving model to ./model\improved.h5
3125/3125 [==============================] - 126s 39ms/step - loss: 3.7975 - accuracy: 0.1240 - val_loss: 3.5728 - val_accuracy: 0.1685
Epoch 2/20
3125/3125 [==============================] - ETA: 0s - loss: 3.1901 - accuracy: 0.2192
Epoch 2: val_loss improved from 3.57285 to 3.01279, saving model to ./model\improved.h5
3125/3125 [==============================] - 121s 39ms/step - loss: 3.1901 - accuracy: 0.2192 - val_loss: 3.0128 - val_accuracy: 0.2534
Epoch 3/20
3125/3125 [==============================] - ETA: 0s - loss: 2.8253 - accuracy: 0.2912
Epoch 3: val_loss improved from 3.01279 to 2.75338, saving model to ./model\improved.h5
3125/3125 [==============================] - 121s 39ms/step - loss: 2.8253 - accuracy: 0.2912 - val_loss: 2.7534 - val_accuracy: 0.2997
Epoch 4/20
3125/3125 [================

In [57]:
mycnn.acc_test('improved')

improved model loss: 1.531700 - acc: 0.580800


In [58]:
atten = True
mycnn.get_model2('ResNet', 100, atten)
mycnn.aug_fit(epoch = 5,atten= atten, model_= 'improved')

Epoch 1/20
3125/3125 [==============================] - ETA: 0s - loss: 3.7906 - accuracy: 0.1168
Epoch 1: val_loss improved from inf to 3.40376, saving model to ./model\improved_atten.h5
3125/3125 [==============================] - 193s 60ms/step - loss: 3.7906 - accuracy: 0.1168 - val_loss: 3.4038 - val_accuracy: 0.1781
Epoch 2/20
3125/3125 [==============================] - ETA: 0s - loss: 3.2003 - accuracy: 0.2107
Epoch 2: val_loss improved from 3.40376 to 2.93828, saving model to ./model\improved_atten.h5
3125/3125 [==============================] - 182s 58ms/step - loss: 3.2003 - accuracy: 0.2107 - val_loss: 2.9383 - val_accuracy: 0.2633
Epoch 3/20
3125/3125 [==============================] - ETA: 0s - loss: 2.8466 - accuracy: 0.2785
Epoch 3: val_loss improved from 2.93828 to 2.64393, saving model to ./model\improved_atten.h5
3125/3125 [==============================] - 192s 61ms/step - loss: 2.8466 - accuracy: 0.2785 - val_loss: 2.6439 - val_accuracy: 0.3227
Epoch 4/20
3125/3125

In [64]:
mycnn.acc_test('attention')

attention model loss: 1.456160 - acc: 0.598900
